In [1]:
import json
# import random
from randomness.Shared_rng import shared_rng
import os
from datetime import datetime
from chunk import Chunk_with_CV
from chunk.Visualize_Operations_Number import visualize_operations_number
from category_integration import Category_Integration
from replace import Replace
from production import Production
from topsim.TopSim import TopSim
from topsim.Spearman_TopSim import Spearman_TopSim
from topsim.Normalized_TopSim import Normalized_TopSim
from topsim.Spearman_Normalized_TopSim import Spearman_Normalized_TopSim
from topsim.Ibuki_1_TopSim import Ibuki_1_TopSim
from topsim.Spearman_Ibuki_1_TopSim import Spearman_Ibuki_1_TopSim
from topsim.Ibuki_2_TopSim import Ibuki_2_TopSim
from topsim.Spearman_Ibuki_2_TopSim import Spearman_Ibuki_2_TopSim
from topsim.Ibuki_3_TopSim import Ibuki_3_TopSim
from topsim.Spearman_Ibuki_3_TopSim import Spearman_Ibuki_3_TopSim
from topsim.Ibuki_4_TopSim import Ibuki_4_TopSim
from topsim.Ibuki_5_TopSim import Ibuki_5_TopSim
from expressivity import Expressivity
from expressivity.Visualize_TopSim_and_Expressivity_CV import visualize_combined_metrics

class Agent:
    def __init__(self, name, learning_algorithms=None):
        self.name = name
        self.memory = []
        self.meanings_for_production = []
        self.production = []
        self.learning_algorithms = learning_algorithms if learning_algorithms else ['chunk', 'category_integration', 'replace']
        self.chunk_application_count = 0
        self.category_integration_count = 0
        self.replace_application_count = 0

        
    def learn_language(self, rule_set, iterations=1, batch_size=1):
        learned_rules = []
        chunk_application_count = 0
        category_integration_count = 0
        replace_application_count = 0

        # print(f"学習開始 - 初期ルールセット: {rule_set}")  # デバッグ出力 # ------------------1115

        for i in range(0, len(rule_set), batch_size):
            batch = rule_set[i:i + batch_size]
            # print(f"\nバッチ {i // batch_size + 1} - バッチ内容: {batch}")  # デバッグ出力 # ------------------1115

            # 累積ルールを次バッチに渡す
            current_batch = learned_rules + batch
            # print(f"累積的なルールセット: {current_batch}")  # デバッグ出力 # ------------------1115

            for iteration in range(iterations):
                # print(f"  Iteration {iteration + 1} 開始")  # デバッグ出力 # ------------------1115
                for algorithm in self.learning_algorithms:
                    if algorithm == 'chunk':
                        current_batch, chunk_applications = Chunk_with_CV.chunk_learning(current_batch)
                        chunk_application_count += chunk_applications
                        # print(f"    チャンク結果: {current_batch}")  # デバッグ出力 # ------------------1115

                    elif algorithm == 'category_integration':
                        current_batch, category_applications = Category_Integration.category_integration_learning(current_batch)
                        category_integration_count += category_applications
                        # print(f"    カテゴリ統合結果: {current_batch}")  # デバッグ出力  # ------------------1115

                    elif algorithm == 'replace':
                        current_batch, replace_applications = Replace.replace_learning(current_batch)
                        replace_application_count += replace_applications
                        # print(f"    置換結果: {current_batch}")  # デバッグ出力  # ------------------1115

            # 学習済みルールを累積更新
            learned_rules = current_batch

        # print(f"\n累積的な学習済みルール: {learned_rules}")  # デバッグ出力  # ------------------1115
        self.memory = learned_rules
        self.chunk_application_count = chunk_application_count
        self.category_integration_count = category_integration_count
        self.replace_application_count = replace_application_count
        # print(f"最終的な学習済みルール: {self.memory}")  # デバッグ出力 # ------------------1115


    def produce_language(self, rule_set, all_meanings, holistic_rule_invention_length, word_rule_invention_length, max_form_length, front_keep_length, shortening_interval, generation):
        self.production = Production.produce(
            rule_set,
            all_meanings,
            holistic_rule_invention_length,
            word_rule_invention_length,
            max_form_length,
            front_keep_length,
            shortening_interval,
            generation
        )
        return self.production

def sample_meaning_space(file_paths):
    meanings_sets = []
    for file_path in file_paths:
        with open(file_path) as f:
            meanings = f.read().split(",\n")
            meanings_sets.append(meanings)
    return meanings_sets

def sample_initial_language(file_paths):
    meanings = []
    for file_path in file_paths:
        with open(file_path) as f:
            s = f.read()
            meanings.extend(s.split(",\n"))
    return meanings


def pad_with_zeros(data_list, target_length):
    return data_list + [0] * (target_length - len(data_list))

def clustering_rule_set(rule_set):
    word_rule_set = []
    sentence_rule_set = []
    
    for rule in rule_set:
        semantic_structure, _ = rule.split('->')  # ルールを意味構造と形式に分割
        if semantic_structure.startswith("S/"):  # 文ルールは"S/"で始まる
            sentence_rule_set.append(rule)
        else:
            word_rule_set.append(rule)
    
    return word_rule_set, sentence_rule_set

def numbering_rule_category(rule_set):
    def parse_rule(rule):
        parts = rule.split('->')
        semantic_structure = parts[0].strip()
        form = parts[1].strip()
        return semantic_structure, form

    holistic_rule_set = []
    generalization_rule_set_1 = []
    generalization_rule_set_2 = []
    generalization_rule_set_3 = []
    word_rule_set = []

    for rule in rule_set:
        semantic_structure, _ = parse_rule(rule)

        if not semantic_structure.startswith("S/"):
            word_rule_set.append(rule)
        else:
            p_count = semantic_structure.count("_p")
            x_count = semantic_structure.count("_x")
            y_count = semantic_structure.count("_y")
            total_variables = p_count + x_count + y_count

            if total_variables == 0:
                holistic_rule_set.append(rule)
            elif total_variables == 1:
                generalization_rule_set_1.append(rule)
            elif total_variables == 2:
                generalization_rule_set_2.append(rule)
            elif total_variables == 3:
                generalization_rule_set_3.append(rule)
                
    len_holistic_rule_set = len(holistic_rule_set)
    len_generalization_rule_set_1 = len(generalization_rule_set_1)
    len_generalization_rule_set_2 = len(generalization_rule_set_2)
    len_generalization_rule_set_3 = len(generalization_rule_set_3)
    len_word_rule_set = len(word_rule_set)
    return len_holistic_rule_set, len_generalization_rule_set_1, len_generalization_rule_set_2, len_generalization_rule_set_3, len_word_rule_set

def remove_synonyms(word_rule_set):
    unique_rules = {}
    for rule in word_rule_set:
        category, meaning, form = rule.split('/')[0], *rule.split('/')[1].split('->')
        key = (category, meaning)  # カテゴリと意味をキーにする
        if key not in unique_rules:
            unique_rules[key] = rule  # 最初に出現したルールだけを保持
    
    return list(unique_rules.values())

def inference_cv_with_probability(child_samples, inference_accurancy_rate):
    inferred_child_samples = []
    success_count = 0
    total_count = len(child_samples)

    for a_child_sample in child_samples:
        sem, form = a_child_sample.split("->")
        original_cv = sem[-1]
        if shared_rng.random() > inference_accurancy_rate:
            inferred_cv = '1' if original_cv == '0' else '0'
        else:
            inferred_cv = original_cv

        if inferred_cv == original_cv:
            success_count += 1

        sem = sem[:-1] + inferred_cv
        an_inferred_child_sample = sem + "->" + form
        inferred_child_samples.append(an_inferred_child_sample)

    # 成功率を計算
    actual_accuracy = success_count / total_count if total_count > 0 else 0
    return inferred_child_samples, actual_accuracy

def simulate_language_evolution(
    n_gens, n_samples, initial_language_files, semantic_space_files,
    holistic_rule_invention_length, word_rule_invention_length, random_seed=1,
    iterations=1, batch_size=1, max_form_length=9, front_keep_length=3,
    shortening_interval=30, inference_accuracy_rate=1.0, base_folder=""
):
    shared_rng.seed(random_seed)

    parent = Agent(name="Parent")
    child = Agent(name="Child", learning_algorithms=['chunk', 'category_integration', 'replace'])
    

    generation_folder = os.path.join(base_folder, "generations")
    os.makedirs(generation_folder, exist_ok=True)
    
    topsim_values_cv0, topsim_values_cv1 = [], []
    topsim_p_values_cv0, topsim_p_values_cv1 = [], []  # 修正箇所: 初期化を追加
    spearman_topsim_values_cv0, spearman_topsim_values_cv1 = [], []
    spearman_p_values_cv0, spearman_p_values_cv1 = [], []  # 修正箇所: 初期化を追加
    
    normalized_topsim_values_cv0, normalized_topsim_values_cv1 = [], []
    normalized_p_values_cv0, normalized_p_values_cv1 = [], []  # 修正箇所: 初期化を追加
    spearman_normalized_topsim_values_cv0, spearman_normalized_topsim_values_cv1 = [], []
    spearman_normalized_p_values_cv0, spearman_normalized_p_values_cv1 = [], []  # 修正箇所: 初期化を追加
    
    ibuki_1_topsim_values_cv0, ibuki_1_topsim_values_cv1 = [], []
    ibuki_1_p_values_cv0, ibuki_1_p_values_cv1 = [], []
    spearman_ibuki_1_topsim_values_cv0, spearman_ibuki_1_topsim_values_cv1 = [], []
    spearman_ibuki_1_p_values_cv0, spearman_ibuki_1_p_values_cv1 = [], []
    
    ibuki_2_topsim_values = []
    ibuki_2_p_values = []  # p値を保存するリストを追加
    spearman_ibuki_2_topsim_values = []
    spearman_ibuki_2_p_values = []  # p値を保存するリストを追加
    
    ibuki_3_topsim_values = []
    ibuki_3_p_values = []  # p値を保存するリストを追加
    spearman_ibuki_3_topsim_values = []
    spearman_ibuki_3_p_values = []  # p値を保存するリストを追加
    
    ibuki_4_topsim_variances = []
    ibuki_4_topsim_averages = []
    ibuki_5_topsim_variances = []
    ibuki_5_topsim_averages = []
    
    expressivity_values_cv0, expressivity_values_cv1 = [0], [0]
    chunk_application_counts = [0]
    category_integration_counts = [0]
    replace_application_counts = [0]
    holistic_rule_counts = [0]
    len_holistic = [0]
    len_gen_1 = [0]
    len_gen_2 = [0]
    len_gen_3 = [0]
    len_word = [0]
    knowledge_counts = [0]
    inference_accuracy_values = []


    # 修正箇所: 最初の世代の言語知識を `semantic_space_files` に基づいて生成
    meaning_spaces = sample_meaning_space(semantic_space_files)
    all_meanings = meaning_spaces[0] + meaning_spaces[1]
    # 修正箇所: 発話アルゴリズムを用いて最初の世代の言語知識を生成
    parent.production = parent.produce_language(
        rule_set=[],  # 初期ルールセットは空に設定
        all_meanings=all_meanings,
        holistic_rule_invention_length=holistic_rule_invention_length,
        word_rule_invention_length=word_rule_invention_length,
        max_form_length=max_form_length,
        front_keep_length=front_keep_length,
        shortening_interval=shortening_interval,
        generation=1
    )
    # print(f"世代1：親の発話: {parent.production}")#------------------------1111
    cv0_production = [x for x in parent.production if "/0->" in x]
    # print("世代1：親の発話のCV_0", cv0_production)
    # print(len(cv0_production))
    cv1_production = [x for x in parent.production if "/1->" in x]
    # print("世代1：親の発話のCV_1", cv1_production)
    # print(len(cv1_production))
    ibuki_2_topsim, ibuki_2_p_value = Ibuki_2_TopSim(parent.production)
    spearman_ibuki_2_topsim, spearman_ibuki_2_p_value = Spearman_Ibuki_2_TopSim(parent.production)
    
    ibuki_3_topsim, ibuki_3_p_value = Ibuki_3_TopSim(parent.production)
    spearman_ibuki_3_topsim, spearman_ibuki_3_p_value = Spearman_Ibuki_3_TopSim(parent.production)
    
    ibuki_4_variance, ibuki_4_average = Ibuki_4_TopSim(parent.production)
    ibuki_5_variance, ibuki_5_average = Ibuki_5_TopSim(parent.production)
    
    child_samples_cv0 = shared_rng.sample(cv0_production, n_samples)
    child_samples_cv1 = shared_rng.sample(cv1_production, n_samples)
    child_samples = child_samples_cv0 + child_samples_cv1
    # print(f"世代1：子の受け取った発話（推論前）: {child_samples}")#------------------------1111
    
    child_samples, success_rate = inference_cv_with_probability(child_samples, inference_accuracy_rate)
    inference_accuracy_values.append(success_rate)
    # print(f"世代1：子の受け取った発話 (推論後): {child_samples}")

    with open(os.path.join(generation_folder, "gen-1.txt"), 'w') as f:
        f.write("\n".join(parent.production))
    with open(os.path.join(generation_folder, "gen-1_child_samples.txt"), 'w') as f:
        f.write("\n".join(child_samples))

    child.learn_language(child_samples, iterations=iterations)
    # **同義語削除を適用**
    word_rules, sentence_rules = clustering_rule_set(child.memory)
    deduplicated_word_rules = remove_synonyms(word_rules)
    child.memory = deduplicated_word_rules + sentence_rules  # 再統合
    len_holistic_rule_set, len_generalization_rule_set_1, len_generalization_rule_set_2, len_generalization_rule_set_3, len_word_rule_set = numbering_rule_category(child.memory)
    with open(os.path.join(generation_folder, "gen-1_child_memory.txt"), 'w') as f:
        f.write("\n".join(child.memory))

    chunk_application_counts.append(child.chunk_application_count)
    category_integration_counts.append(child.category_integration_count)
    replace_application_counts.append(child.replace_application_count)
    len_holistic.append(len_holistic_rule_set)
    len_gen_1.append(len_generalization_rule_set_1)
    len_gen_2.append(len_generalization_rule_set_2)
    len_gen_3.append(len_generalization_rule_set_3)
    len_word.append(len_word_rule_set)
    knowledge_counts.append(len(child.memory))
    # print(f"世代 1 の子の言語知識: {child.memory}") #------------------------1111

    cv0_production = [x for x in parent.production if "/0->" in x]
    cv1_production = [x for x in parent.production if "/1->" in x]

    if cv0_production:
        topsim_cv0, topsim_p_value_cv0 = TopSim(cv0_production)
        topsim_values_cv0.append(topsim_cv0)
        topsim_p_values_cv0.append(topsim_p_value_cv0)
        spearman_topsim_cv0, spearman_p_value_cv0 = Spearman_TopSim(cv0_production)
        spearman_topsim_values_cv0.append(spearman_topsim_cv0)
        spearman_p_values_cv0.append(spearman_p_value_cv0)
        
        normalized_topsim_cv0, normalized_p_value_cv0 = Normalized_TopSim(cv0_production)
        normalized_topsim_values_cv0.append(normalized_topsim_cv0)
        normalized_p_values_cv0.append(normalized_p_value_cv0)
        spearman_normalized_topsim_cv0, spearman_normalized_p_value_cv0 = Spearman_Normalized_TopSim(cv0_production)
        spearman_normalized_topsim_values_cv0.append(spearman_normalized_topsim_cv0)
        spearman_normalized_p_values_cv0.append(spearman_normalized_p_value_cv0)
        
        ibuki_1_topsim_cv0, ibuki_1_p_value_cv0 = Ibuki_1_TopSim(cv0_production)
        ibuki_1_topsim_values_cv0.append(ibuki_1_topsim_cv0)
        ibuki_1_p_values_cv0.append(ibuki_1_p_value_cv0)
        spearman_ibuki_1_topsim_cv0, spearman_ibuki_1_p_value_cv0 = Spearman_Ibuki_1_TopSim(cv0_production)
        spearman_ibuki_1_topsim_values_cv0.append(spearman_ibuki_1_topsim_cv0)
        spearman_ibuki_1_p_values_cv0.append(spearman_ibuki_1_p_value_cv0)
    else:
        topsim_values_cv0.append(0)
        topsim_p_values_cv0.append(None)  # データがない場合は None
        spearman_topsim_values_cv0.append(0)
        spearman_p_values_cv0.append(None)  # データがない場合は None
        
        normalized_topsim_values_cv0.append(0)
        normalized_p_values_cv0.append(None)  # データがない場合は None
        spearman_normalized_topsim_values_cv0.append(0)
        spearman_normalized_p_values_cv0.append(None)  # データがない場合は None
        
        ibuki_1_topsim_values_cv0.append(0)
        ibuki_1_p_values_cv0.append(None)  # データがない場合は None
        spearman_ibuki_1_topsim_values_cv0.append(0)
        spearman_ibuki_1_p_values_cv0.append(None)  # データがない場合は None

    if cv1_production:
        topsim_cv1, topsim_p_value_cv1 = TopSim(cv1_production)
        topsim_values_cv1.append(topsim_cv1)
        topsim_p_values_cv1.append(topsim_p_value_cv1)
        spearman_topsim_cv1, spearman_p_value_cv1 = Spearman_TopSim(cv1_production)
        spearman_topsim_values_cv1.append(spearman_topsim_cv1)
        spearman_p_values_cv1.append(spearman_p_value_cv1)
        
        normalized_topsim_cv1, normalized_p_value_cv1 = Normalized_TopSim(cv1_production)
        normalized_topsim_values_cv1.append(normalized_topsim_cv1)
        normalized_p_values_cv1.append(normalized_p_value_cv1)
        spearman_normalized_topsim_cv1, spearman_normalized_p_value_cv1 = Spearman_Normalized_TopSim(cv1_production)
        spearman_normalized_topsim_values_cv1.append(spearman_normalized_topsim_cv1)
        spearman_normalized_p_values_cv1.append(spearman_normalized_p_value_cv1)
        
        ibuki_1_topsim_cv1, ibuki_1_p_value_cv1 = Ibuki_1_TopSim(cv1_production)
        ibuki_1_topsim_values_cv1.append(ibuki_1_topsim_cv1)
        ibuki_1_p_values_cv1.append(ibuki_1_p_value_cv1)
        spearman_ibuki_1_topsim_cv1, spearman_ibuki_1_p_value_cv1 = Spearman_Ibuki_1_TopSim(cv1_production)
        spearman_ibuki_1_topsim_values_cv1.append(spearman_ibuki_1_topsim_cv1)
        spearman_ibuki_1_p_values_cv1.append(spearman_ibuki_1_p_value_cv1)
    else:
        topsim_values_cv1.append(0)
        topsim_p_values_cv1.append(None)  # データがない場合は None
        spearman_topsim_values_cv1.append(0)
        spearman_p_values_cv1.append(None)  # データがない場合は None
        
        normalized_topsim_values_cv1.append(0)
        normalized_p_values_cv1.append(None)  # データがない場合は None
        spearman_normalized_topsim_values_cv1.append(0)
        spearman_normalized_p_values_cv1.append(None)  # データがない場合は None
        
        ibuki_1_topsim_values_cv1.append(0)
        ibuki_1_p_values_cv1.append(None)  # データがない場合は None
        spearman_ibuki_1_topsim_values_cv1.append(0)
        spearman_ibuki_1_p_values_cv1.append(None)  # データがない場合は None
        
    ibuki_2_topsim_values.append(ibuki_2_topsim)
    ibuki_2_p_values.append(ibuki_2_p_value)  # p値を別リストに保存
    spearman_ibuki_2_topsim_values.append(spearman_ibuki_2_topsim)
    spearman_ibuki_2_p_values.append(spearman_ibuki_2_p_value)  # p値を別リストに保存
    
    ibuki_3_topsim_values.append(ibuki_3_topsim)
    ibuki_3_p_values.append(ibuki_3_p_value)  # p値を別リストに保存
    spearman_ibuki_3_topsim_values.append(spearman_ibuki_3_topsim)
    spearman_ibuki_3_p_values.append(spearman_ibuki_3_p_value)  # p値を別リストに保存
    
    ibuki_4_topsim_variances.append(ibuki_4_variance)
    ibuki_4_topsim_averages.append(ibuki_4_average)
    ibuki_5_topsim_variances.append(ibuki_5_variance)
    ibuki_5_topsim_averages.append(ibuki_5_average)
        
        
# 2世代目以降の処理
    for generation in range(2, n_gens + 1):
        meaning_spaces = sample_meaning_space(semantic_space_files)
        all_meanings = meaning_spaces[0] + meaning_spaces[1]

        parent.production = parent.produce_language(
            rule_set=child.memory,
            all_meanings=all_meanings,
            holistic_rule_invention_length=holistic_rule_invention_length,
            word_rule_invention_length=word_rule_invention_length,
            max_form_length=max_form_length,
            front_keep_length=front_keep_length,
            shortening_interval=shortening_interval,
            generation=generation
        )
        with open(os.path.join(generation_folder, f"gen-{generation}.txt"), 'w') as f:
            f.write("\n".join(parent.production))
        # print(f"世代 {generation} の親の発話: {parent.production}") #------------------------1111

        # child_samples = shared_rng.sample(meaning_spaces[0], n_samples) + shared_rng.sample(meaning_spaces[1], n_samples)
        # cv=0 と cv=1 に分けてからサンプリング
        cv0_production = [x for x in parent.production if "/0->" in x]
        # print(f"世代{generation}の親の発話のCV_0{cv0_production}")
        # print(len(cv0_production))
        cv1_production = [x for x in parent.production if "/1->" in x]
        # print(f"世代{generation}の親の発話のCV_1{cv0_production}")
        # print(len(cv1_production))
        ibuki_2_topsim, ibuki_2_p_value = Ibuki_2_TopSim(parent.production)
        spearman_ibuki_2_topsim, spearman_ibuki_2_p_value = Spearman_Ibuki_2_TopSim(parent.production)
        
        ibuki_3_topsim, ibuki_3_p_value = Ibuki_3_TopSim(parent.production)
        spearman_ibuki_3_topsim, spearman_ibuki_3_p_value = Spearman_Ibuki_3_TopSim(parent.production)
        
        ibuki_4_variance, ibuki_4_average = Ibuki_4_TopSim(parent.production)
        ibuki_5_variance, ibuki_5_average = Ibuki_5_TopSim(parent.production)
        child_samples_cv0 = shared_rng.sample(cv0_production, n_samples)
        child_samples_cv1 = shared_rng.sample(cv1_production, n_samples)
        child_samples = child_samples_cv0 + child_samples_cv1
        # print(f"世代 {generation} の子の受け取った発話（推論前）: {child_samples}")#------------------------1111
        
        child_samples, success_rate = inference_cv_with_probability(child_samples, inference_accuracy_rate)
        inference_accuracy_values.append(success_rate)
        # print(f"世代 {generation} の子の受け取った発話 (推論後): {child_samples}")
        
        with open(os.path.join(generation_folder, f"gen-{generation}_child_samples.txt"), 'w') as f:
            f.write("\n".join(child_samples))

        child.learn_language(child_samples, iterations=iterations)
        # **同義語削除を適用**
        word_rules, sentence_rules = clustering_rule_set(child.memory)
        deduplicated_word_rules = remove_synonyms(word_rules)
        child.memory = deduplicated_word_rules + sentence_rules  # 再統合
        len_holistic_rule_set, len_generalization_rule_set_1, len_generalization_rule_set_2, len_generalization_rule_set_3, len_word_rule_set = numbering_rule_category(child.memory)
        # print(f"世代 {generation} の 子の言語知識: {child.memory}")#------------------------1111
        with open(os.path.join(generation_folder, f"gen-{generation}_child_memory.txt"), 'w') as f:
            f.write("\n".join(child.memory))

        chunk_application_counts.append(child.chunk_application_count)
        category_integration_counts.append(child.category_integration_count)
        replace_application_counts.append(child.replace_application_count)
        len_holistic.append(len_holistic_rule_set)
        len_gen_1.append(len_generalization_rule_set_1)
        len_gen_2.append(len_generalization_rule_set_2)
        len_gen_3.append(len_generalization_rule_set_3)
        len_word.append(len_word_rule_set)
        knowledge_counts.append(len(child.memory))

        # cv0_production = [x for x in parent.production if x.endswith("/0")]
        # print('これです！cv0_性生物', cv0_production)
        # cv1_production = [x for x in parent.production if x.endswith("/1")]

        if cv0_production:
            topsim_cv0, topsim_p_value_cv0 = TopSim(cv0_production)
            topsim_values_cv0.append(topsim_cv0)
            topsim_p_values_cv0.append(topsim_p_value_cv0)
            spearman_topsim_cv0, spearman_p_value_cv0 = Spearman_TopSim(cv0_production)
            spearman_topsim_values_cv0.append(spearman_topsim_cv0)
            spearman_p_values_cv0.append(spearman_p_value_cv0)
            
            normalized_topsim_cv0, normalized_p_value_cv0 = Normalized_TopSim(cv0_production)
            normalized_topsim_values_cv0.append(normalized_topsim_cv0)
            normalized_p_values_cv0.append(normalized_p_value_cv0)
            spearman_normalized_topsim_cv0, spearman_normalized_p_value_cv0 = Spearman_Normalized_TopSim(cv0_production)
            spearman_normalized_topsim_values_cv0.append(spearman_normalized_topsim_cv0)
            spearman_normalized_p_values_cv0.append(spearman_normalized_p_value_cv0)
            
            ibuki_1_topsim_cv0, ibuki_1_p_value_cv0 = Ibuki_1_TopSim(cv0_production)
            ibuki_1_topsim_values_cv0.append(ibuki_1_topsim_cv0)
            ibuki_1_p_values_cv0.append(ibuki_1_p_value_cv0)
            spearman_ibuki_1_topsim_cv0, spearman_ibuki_1_p_value_cv0 = Spearman_Ibuki_1_TopSim(cv0_production)
            spearman_ibuki_1_topsim_values_cv0.append(spearman_ibuki_1_topsim_cv0)
            spearman_ibuki_1_p_values_cv0.append(spearman_ibuki_1_p_value_cv0)
        else:
            topsim_values_cv0.append(0)
            topsim_p_values_cv0.append(None)  # データがない場合は None
            spearman_topsim_values_cv0.append(0)
            spearman_p_values_cv0.append(None)  # データがない場合は None
            
            normalized_topsim_values_cv0.append(0)
            normalized_p_values_cv0.append(None)  # データがない場合は None
            spearman_normalized_topsim_values_cv0.append(0)
            spearman_normalized_p_values_cv0.append(None)  # データがない場合は None
            
            ibuki_1_topsim_values_cv0.append(0)
            ibuki_1_p_values_cv0.append(None)  # データがない場合は None
            spearman_ibuki_1_topsim_values_cv0.append(0)
            spearman_ibuki_1_p_values_cv0.append(None)  # データがない場合は None

        if cv1_production:
            topsim_cv1, topsim_p_value_cv1 = TopSim(cv1_production)
            topsim_values_cv1.append(topsim_cv1)
            topsim_p_values_cv1.append(topsim_p_value_cv1)
            spearman_topsim_cv1, spearman_p_value_cv1 = Spearman_TopSim(cv1_production)
            spearman_topsim_values_cv1.append(spearman_topsim_cv1)
            spearman_p_values_cv1.append(spearman_p_value_cv1)
            
            normalized_topsim_cv1, normalized_p_value_cv1 = Normalized_TopSim(cv1_production)
            normalized_topsim_values_cv1.append(normalized_topsim_cv1)
            normalized_p_values_cv1.append(normalized_p_value_cv1)
            spearman_normalized_topsim_cv1, spearman_normalized_p_value_cv1 = Spearman_Normalized_TopSim(cv1_production)
            spearman_normalized_topsim_values_cv1.append(spearman_normalized_topsim_cv1)
            spearman_normalized_p_values_cv1.append(spearman_normalized_p_value_cv1)
            
            ibuki_1_topsim_cv1, ibuki_1_p_value_cv1 = Ibuki_1_TopSim(cv1_production)
            ibuki_1_topsim_values_cv1.append(ibuki_1_topsim_cv1)
            ibuki_1_p_values_cv1.append(ibuki_1_p_value_cv1)
            spearman_ibuki_1_topsim_cv1, spearman_ibuki_1_p_value_cv1 = Spearman_Ibuki_1_TopSim(cv1_production)
            spearman_ibuki_1_topsim_values_cv1.append(spearman_ibuki_1_topsim_cv1)
            spearman_ibuki_1_p_values_cv1.append(spearman_ibuki_1_p_value_cv1)
        else:
            topsim_values_cv1.append(0)
            topsim_p_values_cv1.append(None)  # データがない場合は None
            spearman_topsim_values_cv1.append(0)
            spearman_p_values_cv1.append(None)  # データがない場合は None
            
            normalized_topsim_values_cv1.append(0)
            normalized_p_values_cv1.append(None)  # データがない場合は None
            spearman_normalized_topsim_values_cv1.append(0)
            spearman_normalized_p_values_cv1.append(None)  # データがない場合は None
            
            ibuki_1_topsim_values_cv1.append(0)
            ibuki_1_p_values_cv1.append(None)  # データがない場合は None
            spearman_ibuki_1_topsim_values_cv1.append(0)
            spearman_ibuki_1_p_values_cv1.append(None)  # データがない場合は None
            
        ibuki_2_topsim_values.append(ibuki_2_topsim)
        ibuki_2_p_values.append(ibuki_2_p_value)  # p値を別リストに保存
        spearman_ibuki_2_topsim_values.append(spearman_ibuki_2_topsim)
        spearman_ibuki_2_p_values.append(spearman_ibuki_2_p_value)  # p値を別リストに保存
        
        ibuki_3_topsim_values.append(ibuki_3_topsim)
        ibuki_3_p_values.append(ibuki_3_p_value)  # p値を別リストに保存
        spearman_ibuki_3_topsim_values.append(spearman_ibuki_3_topsim)
        spearman_ibuki_3_p_values.append(spearman_ibuki_3_p_value)  # p値を別リストに保存
        
        ibuki_4_topsim_variances.append(ibuki_4_variance)
        ibuki_4_topsim_averages.append(ibuki_4_average)
        ibuki_5_topsim_variances.append(ibuki_5_variance)
        ibuki_5_topsim_averages.append(ibuki_5_average)

        expressivity_values_cv0.append(Expressivity.expressivity(child.memory, [x for x in all_meanings if x.endswith("/0")]))
        expressivity_values_cv1.append(Expressivity.expressivity(child.memory, [x for x in all_meanings if x.endswith("/1")]))
    
    topsim_values_cv0 = pad_with_zeros(topsim_values_cv0, n_gens)
    topsim_values_cv1 = pad_with_zeros(topsim_values_cv1, n_gens)
    topsim_p_values_cv0 = pad_with_zeros(topsim_p_values_cv0, n_gens)
    topsim_p_values_cv1 = pad_with_zeros(topsim_p_values_cv1, n_gens)
    spearman_topsim_values_cv0 = pad_with_zeros(spearman_topsim_values_cv0, n_gens)
    spearman_topsim_values_cv1 = pad_with_zeros(spearman_topsim_values_cv1, n_gens)
    spearman_p_values_cv0 = pad_with_zeros(spearman_p_values_cv0, n_gens)
    spearman_p_values_cv1 = pad_with_zeros(spearman_p_values_cv1, n_gens)
    
    normalized_topsim_values_cv0 = pad_with_zeros(normalized_topsim_values_cv0, n_gens)
    normalized_topsim_values_cv1 = pad_with_zeros(normalized_topsim_values_cv1, n_gens)
    normalized_p_values_cv0 = pad_with_zeros(normalized_p_values_cv0, n_gens)
    normalized_p_values_cv1 = pad_with_zeros(normalized_p_values_cv1, n_gens)
    spearman_normalized_topsim_values_cv0 = pad_with_zeros(spearman_normalized_topsim_values_cv0, n_gens)
    spearman_normalized_topsim_values_cv1 = pad_with_zeros(spearman_normalized_topsim_values_cv1, n_gens)
    spearman_normalized_p_values_cv0 = pad_with_zeros(spearman_normalized_p_values_cv0, n_gens)
    spearman_normalized_p_values_cv1 = pad_with_zeros(spearman_normalized_p_values_cv1, n_gens)
    
    ibuki_1_topsim_values_cv0 = pad_with_zeros(ibuki_1_topsim_values_cv0, n_gens)
    ibuki_1_topsim_values_cv1 = pad_with_zeros(ibuki_1_topsim_values_cv1, n_gens)
    ibuki_1_p_values_cv0 = pad_with_zeros(ibuki_1_p_values_cv0, n_gens)
    ibuki_1_p_values_cv1 = pad_with_zeros(ibuki_1_p_values_cv1, n_gens)
    spearman_ibuki_1_topsim_values_cv0 = pad_with_zeros(spearman_ibuki_1_topsim_values_cv0, n_gens)
    spearman_ibuki_1_topsim_values_cv1 = pad_with_zeros(spearman_ibuki_1_topsim_values_cv1, n_gens)
    spearman_ibuki_1_p_values_cv0 = pad_with_zeros(spearman_ibuki_1_p_values_cv0, n_gens)
    spearman_ibuki_1_p_values_cv1 = pad_with_zeros(spearman_ibuki_1_p_values_cv1, n_gens)
    
    ibuki_2_topsim_values = pad_with_zeros(ibuki_2_topsim_values, n_gens)
    ibuki_2_p_values = pad_with_zeros(ibuki_2_p_values, n_gens)  # p値の補完を追加
    spearman_ibuki_2_topsim_values = pad_with_zeros(spearman_ibuki_2_topsim_values, n_gens)
    spearman_ibuki_2_p_values = pad_with_zeros(spearman_ibuki_2_p_values, n_gens)  # Spearmanのp値の補完を追加
    
    ibuki_3_topsim_values = pad_with_zeros(ibuki_3_topsim_values, n_gens)
    ibuki_3_p_values = pad_with_zeros(ibuki_3_p_values, n_gens)  # p値の補完を追加
    spearman_ibuki_3_topsim_values = pad_with_zeros(spearman_ibuki_3_topsim_values, n_gens)
    spearman_ibuki_3_p_values = pad_with_zeros(spearman_ibuki_3_p_values, n_gens)  # Spearmanのp値の補完を追加
    
    ibuki_4_topsim_variances = pad_with_zeros(ibuki_4_topsim_variances, n_gens)
    ibuki_4_topsim_averages = pad_with_zeros(ibuki_4_topsim_averages, n_gens)
    ibuki_5_topsim_variances = pad_with_zeros(ibuki_5_topsim_variances, n_gens)
    ibuki_5_topsim_averages = pad_with_zeros(ibuki_5_topsim_averages, n_gens)
    expressivity_values_cv0 = pad_with_zeros(expressivity_values_cv0, n_gens)
    expressivity_values_cv1 = pad_with_zeros(expressivity_values_cv1, n_gens)

    output_path_pearson_metrics = os.path.join(base_folder, "pearson_metrics.png")
    output_path_spearman_metrics = os.path.join(base_folder, "spearman_metrics.png")
    output_path_pearson_ibuki_metrics = os.path.join(base_folder, "pearson_ibuki_metrics.png")
    output_path_spearman_ibuki_metrics = os.path.join(base_folder, "spearman_ibuki_metrics.png")
    output_path_ibuki_metrics = os.path.join(base_folder, "ibuki_metrics_variance_average.png")
    output_path_inference = os.path.join(base_folder, "inference_accuracy.png")
    output_path_ibuki_index_variance_similar_pearson = os.path.join(base_folder, "ibuki_index_variance_similar_pearson.png")
    output_path_ibuki_index_variance_similar_spearman = os.path.join(base_folder, "ibuki_index_variance_similar_spearman.png")
    output_path_ibuki_index_average_similar_pearson = os.path.join(base_folder, "ibuki_index_average_similar_pearson.png")
    output_path_ibuki_index_average_similar_spearman = os.path.join(base_folder, "ibuki_index_average_similar_spearman.png")
    output_path_ibuki_set_variance_difference_pearson = os.path.join(base_folder, "ibuki_set_variance_difference_pearson.png")
    output_path_ibuki_set_variance_difference_spearman = os.path.join(base_folder, "ibuki_set_variance_difference_spearman.png")
    output_path_ibuki_set_average_difference_pearson = os.path.join(base_folder, "ibuki_set_average_difference_pearson.png")
    output_path_ibuki_set_average_difference_spearman = os.path.join(base_folder, "ibuki_set_average_difference_spearman.png")


    
    visualize_combined_metrics(
        topsim_values_cv0, topsim_values_cv1,
        spearman_topsim_values_cv0, spearman_topsim_values_cv1,
        
        normalized_topsim_values_cv0, normalized_topsim_values_cv1,
        spearman_normalized_topsim_values_cv0, spearman_normalized_topsim_values_cv1,
        
        ibuki_1_topsim_values_cv0, ibuki_1_topsim_values_cv1,
        spearman_ibuki_1_topsim_values_cv0, spearman_ibuki_1_topsim_values_cv1,
        
        expressivity_values_cv0, expressivity_values_cv1,
        inference_accuracy_values,
        ibuki_2_topsim_values, 
        spearman_ibuki_2_topsim_values, 
        
        ibuki_3_topsim_values,
        spearman_ibuki_3_topsim_values, 
        
        ibuki_4_topsim_variances, ibuki_4_topsim_averages,
        ibuki_5_topsim_variances, ibuki_5_topsim_averages,
        n_gens, 
        output_path_pearson_metrics,
        output_path_spearman_metrics,
        output_path_pearson_ibuki_metrics,
        output_path_spearman_ibuki_metrics,
        output_path_ibuki_metrics,
        output_path_inference,  # 新たに追加
        output_path_ibuki_index_variance_similar_pearson,  # Index 分散 (Pearson)
        output_path_ibuki_index_average_similar_pearson,  # Index 平均 (Pearson)
        output_path_ibuki_set_variance_difference_pearson, # Set 分散 (Pearson)
        output_path_ibuki_set_average_difference_pearson,  # Set 平均 (Pearson)
        output_path_ibuki_index_variance_similar_spearman,  # Index 分散 (Spearman)
        output_path_ibuki_index_average_similar_spearman,  # Index 平均 (Spearman)
        output_path_ibuki_set_variance_difference_spearman, # Set 分散 (Spearman)
        output_path_ibuki_set_average_difference_spearman  # Set 平均 (Spearman)
    )

    # それぞれのグラフの保存先を定義
    out_path_3_learnings_and_knowledge_counts = os.path.join(base_folder, "3_learnings_and_knowledge_counts.png")
    out_path_composition_of_knowledge = os.path.join(base_folder, "composition_of_knowledge.png")
    out_path_knowledge_counts_and_composition = os.path.join(base_folder, "knowledge_counts_and_composition.png")

    # 3つのグラフの出力を実行
    visualize_operations_number(
        chunk_application_counts, category_integration_counts, replace_application_counts,
        knowledge_counts, len_holistic, len_gen_1, len_gen_2, len_gen_3, len_word,
        n_gens,
        [out_path_3_learnings_and_knowledge_counts, out_path_composition_of_knowledge, out_path_knowledge_counts_and_composition],  # 保存先パスをリストで渡す
        N_SAMPLES
    )

    metrics = {
        "TopSim_CV0": spearman_topsim_values_cv0,
        "TopSim_p_Value_CV0": spearman_p_values_cv0,  # p値を追加
        "TopSim_CV1": spearman_topsim_values_cv1,
        "TopSim_p_Value_CV1": spearman_p_values_cv1,  # p値を追加
        "Spearman_TopSim_CV0": spearman_topsim_values_cv0,
        "Spearman_p_Value_CV0": spearman_p_values_cv0,  # p値を追加
        "Spearman_TopSim_CV1": spearman_topsim_values_cv1,
        "Spearman_p_Value_CV1": spearman_p_values_cv1,  # p値を追加
        
        "Normalized_TopSim_CV0": normalized_topsim_values_cv0,
        "Normalized_p_Value_CV0": normalized_p_values_cv0,  # p値を追加
        "Normalized_TopSim_CV1": normalized_topsim_values_cv1,
        "Normalized_p_Value_CV1": normalized_p_values_cv1,  # p値を追加
        "Spearman_Normalized_TopSim_CV0": spearman_normalized_topsim_values_cv0,
        "Spearman_Normalized_p_Value_CV0": spearman_normalized_p_values_cv0,  # p値を追加
        "Spearman_Normalized_TopSim_CV1": spearman_normalized_topsim_values_cv1,
        "Spearman_Normalized_p_Value_CV1": spearman_normalized_p_values_cv1,  # p値を追加
        
        "Ibuki_1_TopSim_CV0": ibuki_1_topsim_values_cv0,
        "Ibuki_1_p_Value_CV0": ibuki_1_p_values_cv0,  # p値を追加
        "Ibuki_1_TopSim_CV1": ibuki_1_topsim_values_cv1,
        "Ibuki_1_p_Value_CV1": ibuki_1_p_values_cv1,  # p値を追加
        "Spearman_Ibuki_1_TopSim_CV0": spearman_ibuki_1_topsim_values_cv0,
        "Spearman_Ibuki_1_p_Value_CV0": spearman_ibuki_1_p_values_cv0,  # p値を追加
        "Spearman_Ibuki_1_TopSim_CV1": spearman_ibuki_1_topsim_values_cv1,
        "Spearman_Ibuki_1_p_Value_CV1": spearman_ibuki_1_p_values_cv1,  # p値を追加
        
        "Ibuki_2_TopSim": ibuki_2_topsim_values,
        "Ibuki_2_p_Values": ibuki_2_p_values,  # p値を追加保存
        "Spearman_Ibuki_2_TopSim": spearman_ibuki_2_topsim_values,
        "Spearman_Ibuki_2_p_Values": spearman_ibuki_2_p_values,  # p値を追加保存
        
        "Ibuki_3_TopSim": ibuki_3_topsim_values,
        "Ibuki_3_p_Values": ibuki_3_p_values,  # p値を追加保存
        "Spearman_Ibuki_3_TopSim": spearman_ibuki_3_topsim_values,
        "Spearman_Ibuki_3_p_Values": spearman_ibuki_3_p_values,  # p値を追加保存
        
        "Ibuki_4_TopSim_Variances": ibuki_4_topsim_variances,
        "Ibuki_4_TopSim_Averages": ibuki_4_topsim_averages,
        "Ibuki_5_TopSim_Variances": ibuki_5_topsim_variances,
        "Ibuki_5_TopSim_Averages": ibuki_5_topsim_averages,
        "Expressivity_CV0": expressivity_values_cv0,
        "Expressivity_CV1": expressivity_values_cv1,
        "Inference_Accuracy": inference_accuracy_values,
        "Knowledge_Counts": knowledge_counts,
        "Holistic_Rules": len_holistic,
        "Generalization_Rules_1": len_gen_1,
        "Generalization_Rules_2": len_gen_2,
        "Generalization_Rules_3": len_gen_3,
        "Word_Rules": len_word
    }

    with open(os.path.join(base_folder, "TopSim_and_Expressivity_Values.json"), 'w') as f:
        json.dump(metrics, f, indent=4)

    return parent.production, child_samples, child.memory, topsim_values_cv0, spearman_topsim_values_cv0, normalized_topsim_values_cv0, spearman_normalized_topsim_values_cv0, ibuki_1_topsim_values_cv0, spearman_ibuki_1_topsim_values_cv0, expressivity_values_cv0, chunk_application_counts, category_integration_counts, replace_application_counts

if __name__ == "__main__":
    N_GENS = 400
    N_SAMPLES = 24
    INITIAL_LANGUAGE_FILES = ["data/48_Initial_Language_0.txt", "data/48_Initial_Language_1.txt"]
    SEMANTIC_SPACE_FILES = ["data/48_Semantic_Space_0.txt", "data/48_Semantic_Space_1.txt"]
    HOLISTIC_RULE_INVENTION_LENGTH = 6
    WORD_RULE_INVENTION_LENGTH = 1
    ITERATIONS = 1
    BATCH_SIZE = 1
    MAX_FORM_LENGTH = 9
    FRONT_KEEP_LENGTH = 6
    SHORTENING_INTERVAL = 10
    INFERENCE_ACCURACY_RATE = 1.0
    SEED_MAX = 100
    
    # 修正箇所：フォルダ名の生成部分    
    formatted_inference_rate = f"{int(INFERENCE_ACCURACY_RATE * 10):02d}"  # 推論成功確率を整数に変換し09のような形式に
    formatted_n_gens = f"gen{N_GENS}"  # 世代数
    formatted_n_samples = f"sample{N_SAMPLES}"  # サンプル数
    
    now = datetime.now()
    timestamp = now.strftime('%Y%m%d_%H%M%S')
    base_folder_name = f"exp{timestamp}_{formatted_n_gens}_infer{formatted_inference_rate}_{formatted_n_samples}"
    base_folder = os.path.join("out", base_folder_name)
    os.makedirs(base_folder, exist_ok=True)
    

    for seed in range(1, SEED_MAX + 1):
        # Seed に応じたサブフォルダを作成
        seed_folder_name = f"{base_folder_name}_seed{seed}"
        seed_folder = os.path.join(base_folder, seed_folder_name)
        os.makedirs(seed_folder, exist_ok=True)
        
        # print(f"Experiment with seed: {seed}")
        

        parameters = {
            "N_GENS": N_GENS,
            "N_SAMPLES": N_SAMPLES,
            "INITIAL_LANGUAGE_FILES": INITIAL_LANGUAGE_FILES,
            "SEMANTIC_SPACE_FILES": SEMANTIC_SPACE_FILES,
            "HOLISTIC_RULE_INVENTION_LENGTH": HOLISTIC_RULE_INVENTION_LENGTH,
            "WORD_RULE_INVENTION_LENGTH": WORD_RULE_INVENTION_LENGTH,
            "ITERATIONS": ITERATIONS,
            "BATCH_SIZE": BATCH_SIZE,
            "MAX_FORM_LENGTH": MAX_FORM_LENGTH,
            "FRONT_KEEP_LENGTH": FRONT_KEEP_LENGTH,
            "SHORTENING_INTERVAL": SHORTENING_INTERVAL,
            "SEED_MAX": SEED_MAX,
            "INFERENCE_ACCURACY_RATE":INFERENCE_ACCURACY_RATE,
            "SEED": seed,
        }
        with open(os.path.join(base_folder, "states.json"), 'w') as f:
            json.dump(parameters, f, indent=4)

        parent_production, child_samples, child_memory, topsim_values, spearman_topsim_values, normalized_topsim_values, spearman_normalized_topsim_values, ibuki_1_topsim_values, spearman_ibuki_1_topsim_values, expressivity_values, chunk_application_counts, category_integration_counts, replace_application_counts = simulate_language_evolution(
            n_gens=N_GENS, 
            n_samples=N_SAMPLES, 
            initial_language_files=INITIAL_LANGUAGE_FILES,
            semantic_space_files=SEMANTIC_SPACE_FILES,
            holistic_rule_invention_length=HOLISTIC_RULE_INVENTION_LENGTH,
            word_rule_invention_length=WORD_RULE_INVENTION_LENGTH,
            random_seed=seed,
            iterations=ITERATIONS,
            batch_size=BATCH_SIZE,
            max_form_length=MAX_FORM_LENGTH,
            front_keep_length=FRONT_KEEP_LENGTH,
            shortening_interval=SHORTENING_INTERVAL,
            inference_accuracy_rate=INFERENCE_ACCURACY_RATE,
            base_folder=seed_folder
        )
        
        # print(f"最終世代の発話seed{seed}:", parent_production)
        # print(f"最終世代の子供が受け取った発話seed{seed}:", child_samples)
        # print(f"最終世代の子供の記憶seed{seed}:", child_memory)
        # print(f"各世代のTopSim値seed{seed}:", topsim_values)
        # print(f"各世代のNormalized_TopSim値seed{seed}:", normalized_topsim_values)
        # print(f"各世代のIbuki_1_TopSim値seed{seed}:", ibuki_1_topsim_values)
        # print(f"各世代のExpressivity値seed{seed}:", expressivity_values)
        # print(f"各世代のチャンク適用回数seed{seed}:", chunk_application_counts)
        # print(f"各世代のカテゴリ統合適用回数seed{seed}:", category_integration_counts)
        # print(f"各世代のリプレース適用回数seed{seed}:", replace_application_counts)
        # print("-" * 50)